In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', 500)

In [2]:
# Features - Description

# title - Name of the product
# Rating- average rating given to a product
# maincateg - category that the product is listed under(men/women)
# platform - platform on which it is sold on (Eg. Amazon, Flipkart)
# price1 - Discounted Price of the listed product
# actprice1 - Actual price of the listed product
# Offer % - Discount percent
# norating1 - number of ratings available for a particular product
# noreviews1 - number of reviews available for a particular product
# star_5f - number of five star ratings given to a particular product
# star_4f - number of four star ratings given to a particular product
# star_3f - number of three star ratings given to a particular product
# star_2f - number of two star ratings given to a particular product
# star_1f - number of one star ratings given to a particular product
# fulfilled1- whether it is Amazon fulfilled or not

In [3]:
data = pd.read_csv( '../input/sa2022/train.csv' )
data_target = data[['price1']]
data_id = data[['id']]
data.drop(columns=['id','price1'],inplace=True)

In [4]:
df_Test = pd.read_csv( '../input/sa2022/test.csv' )
df_Test_id = df_Test[['id']]
df_Test.drop(columns=['id'],inplace=True)

In [5]:
# sum_star = data['star_5f'] + data['star_4f'] + data['star_3f'] +data['star_2f'] + data['star_1f']
# data['norating1'] - sum_star

# # Number ratings is equal to sum of number of stars

In [6]:
data

In [7]:
data.isnull().sum()

In [8]:
df_Test.isnull().sum()

***
Applying CountVecterizer on 

In [9]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# # Cleaning the texts
# import re
# from nltk.corpus import stopwords

In [10]:
# list_sentences = []
# for id in data.index.values:
#     sentence = data.at[id,'title']
#     list_sentences.append( sentence )
    

# corpus = []
# for i in range(len(list_sentences)):
#     review = re.sub('[^a-zA-Z]', ' ', list_sentences[i])
#     review = review.lower()
#     review = review.split()
#     review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
#     review = ' '.join(review)
#     corpus.append(review)
    
# # Creating the Bag of Words model
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features=100)
# cv.fit(corpus)

# X = cv.transform(corpus).toarray()

# words_df = pd.DataFrame( X )
# words_df.head(5)

# data = pd.concat([data,words_df],axis=1)

In [11]:
# list_sentences = []
# for id in df_Test.index.values:
#     sentence = df_Test.at[id,'title']
#     list_sentences.append( sentence )

# corpus = []
# for i in range(len(list_sentences)):
#     review = re.sub('[^a-zA-Z]', ' ', list_sentences[i])
#     review = review.lower()
#     review = review.split()
#     review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
#     review = ' '.join(review)
#     corpus.append(review)
    
# X = cv.transform(corpus).toarray()

# words_df = pd.DataFrame( X )
# words_df.head(5)

# df_Test = pd.concat([df_Test,words_df],axis=1)

Applying Bag of Words on "title" features didn't improves the accuracy
***

In [12]:
data.platform.unique(), data.maincateg.unique() 

In [13]:
categorical_features_name = ['maincateg','platform','fulfilled1']
numerical_features_name = list( set(data.columns.values) - set(categorical_features_name) - set(['Offer %','title']) )

In [14]:
# i=0
# data_num = data[numerical_features_name]
# fig = make_subplots( rows=1, cols = data_num.shape[1],subplot_titles = data_num.columns.values )
# for feature in data_num.columns.values:
#   fig.add_trace( go.Histogram( x=data_num[feature], name=feature), row=1,col=i+1 )
#   i = i+1

# fig.update_layout( width=10000 )
# fig.show()

In [15]:
# i=0
# data_num = data[numerical_features_name]
# fig = make_subplots( rows=1, cols = data_num.shape[1],subplot_titles = data_num.columns.values )
# for feature in data_num.columns.values:
#   fig.add_trace( go.Box( y=data_num[feature], name=feature), row=1,col=i+1 )
#   i = i+1

# fig.update_layout( width=5500, height=700 )
# fig.show()

In [16]:
# Imputing Missing values in "maincateg" using "title" columns
# Data
data.maincateg.fillna(value='NaN',inplace=True)
df_temp = data[ data.maincateg == "NaN" ]

for id in df_temp.index.values:
    sentence = data.at[id,'title']
    find_result_women = sentence.find('Women')
    find_result_men = sentence.find('Men')
    
    if find_result_men != -1:
        value = "Men"
    if find_result_women != -1:
        value = "Women"
        
    data.at[id,'maincateg'] = value

In [17]:
# Imputing Missing values in "maincateg" using "title" columns
# df_Test
df_Test.maincateg.fillna(value='NaN',inplace=True)
df_temp = df_Test[ df_Test.maincateg == "NaN" ]

for id in df_temp.index.values:
    sentence = data.at[id,'title']
    find_result_women = sentence.find('Women')
    find_result_men = sentence.find('Men')
    
    if find_result_men != -1:
        value = "Men"
    if find_result_women != -1:
        value = "Women"
        
    data.at[id,'maincateg'] = value

In [18]:
from sklearn.preprocessing import OneHotEncoder

def One_Hot_Encoder (df,feature): # Binary Category
  ohe = OneHotEncoder(sparse=False,drop='first',handle_unknown='ignore')
  ohe.fit( df[[feature]] )
  temp = ohe.transform( df[[feature]] )
  temp = pd.DataFrame( temp, columns=[feature] )
  df.drop(columns=feature,axis=1,inplace=True)
  df = pd.concat( [df,temp],axis=1 )
  return df, ohe

In [19]:
data, ohe_maincateg = One_Hot_Encoder(data,feature='maincateg')

feature = 'maincateg'
temp = ohe_maincateg.transform( df_Test[[feature]] )
temp = pd.DataFrame( temp, columns=[feature] )
df_Test.drop(columns=feature,axis=1,inplace=True)
df_Test = pd.concat( [df_Test,temp],axis=1 )
df_Test

In [20]:
data, ohe_platform = One_Hot_Encoder(data,feature='platform')

feature = 'platform'
temp = ohe_platform.transform( df_Test[[feature]] )
temp = pd.DataFrame( temp, columns=[feature] )
df_Test.drop(columns=feature,axis=1,inplace=True)
df_Test = pd.concat( [df_Test,temp],axis=1 )
df_Test

In [21]:
data.isnull().sum()

In [22]:
df_Test.isnull().sum()

In [23]:
def simple_imputer_mode(df, feature):
    simple_imputer_mode = SimpleImputer(strategy='most_frequent')
    simple_imputer_mode.fit(df[[feature]])
    df[feature] = simple_imputer_mode.transform(df[[feature]])
    return df, simple_imputer_mode

In [24]:
data, imputer_star_5f = simple_imputer_mode(data,feature='star_5f')

df_Test['star_5f'] = imputer_star_5f.transform( df_Test[['star_5f']] )

data, imputer_star_4f = simple_imputer_mode(data,feature='star_4f')
data, imputer_star_3f = simple_imputer_mode(data,feature='star_3f')

In [25]:
feature = 'star_1f'
simple_imputer_mode_star_1f = SimpleImputer(strategy='most_frequent')
simple_imputer_mode_star_1f.fit(data[[feature]])
df_Test[feature] = simple_imputer_mode_star_1f.transform(df_Test[[feature]])


feature = 'Rating'
simple_imputer_mode_Rating = SimpleImputer(strategy='mean')
simple_imputer_mode_Rating.fit(data[[feature]])
df_Test[feature] = simple_imputer_mode_Rating.transform(df_Test[[feature]])

In [26]:
data, imputer_norating1 = simple_imputer_mode(data,feature='norating1')
data, imputer_noreviews1 = simple_imputer_mode(data,feature='noreviews1')

In [27]:
from sklearn.model_selection import train_test_split
X = data.drop(columns=['title','Offer %'])
y = data_target['price1']
X_train, X_test, y_train ,y_test = train_test_split( X, y, test_size=0.25,  random_state=42 ) 

In [28]:
# from sklearn.preprocessing import StandardScaler
# RS = StandardScaler()
# RS.fit(X_train[numerical_features_name])
# X_train[numerical_features_name] = RS.transform(X_train[numerical_features_name])
# X_test[numerical_features_name] = RS.transform( X_test[numerical_features_name] )

In [29]:
X_train.head()

In [30]:
# Metric
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

RMSE_test = []
RMSE_train = []
Model_Name = []

In [31]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge


model_list = [RandomForestRegressor(),XGBRegressor(),DecisionTreeRegressor(),ElasticNet(), # SVR(),
              GradientBoostingRegressor(),ExtraTreesRegressor(),
              AdaBoostRegressor(),Lasso(),LinearRegression(),Ridge()]

for model in model_list:
    print(model)
    model.fit(X_train, y_train)

    print('Test Set --------------')
    y_predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, y_predictions)
    rmse = np.sqrt(mse)
    score = model.score(X_test,y_test)

    RMSE_test.append(rmse)
    Model_Name.append( 'RandomForestRegressor' )

    print('Root Mean Square Error : ', rmse )
    print('R2 Score : ', score )
    print('\n')

    print('Train Set --------------')
    y_predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, y_predictions)
    rmse = np.sqrt(mse)
    score = model.score(X_train,y_train)
    RMSE_train.append(rmse)
    print('Root Mean Square Error : ', rmse ) 
    print('R2 Score : ', score )
    
    print('\n')
    print('\n')
    print("-------------------------------------------------------------------------------")

In [32]:
pd.set_option('display.max_rows', 15)
results = pd.DataFrame()
results['Model_Name'] = model_list
results['RMSE on Test Set'] = RMSE_test
results['RMSE on Train Set'] = RMSE_train
results = results.sort_values(by='RMSE on Test Set')
results

## Hypertuning

In [33]:
params = {'bootstrap': [True, False],
             'max_depth': [2,3,5,6],
             # ax_features': ['auto', 'sqrt'],
             'min_samples_leaf': [1,2,3,4],
             'min_samples_split': [1,2,5],
             'n_estimators': [50,100,500]
         }

model = RandomForestRegressor() 
GS_CV = GridSearchCV(estimator=model, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=3)
GS_CV.fit(X_train, y_train)
print("Best parameters:", GS_CV.best_params_)

Best parameters: {'bootstrap': True, 'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
<br>
<br>
Using these 'Best parameters' model is not overfitting, we will use this model as our final model

In [71]:
model = RandomForestRegressor(bootstrap=True, max_depth=6, min_samples_leaf=4, min_samples_split=2, n_estimators=100) 

model.fit(X_train, y_train)

print('Test Set --------------')
y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

print('Root Mean Square Error : ', rmse )
print('R2 Score : ', score )
print('\n')

print('Train Set --------------')
y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmse = np.sqrt(mse)
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
print('Root Mean Square Error : ', rmse ) 
print('R2 Score : ', score )
print("-------------------------------------------------------------------------------")

In [35]:
GBR=GradientBoostingRegressor()
from sklearn.model_selection import GridSearchCV
search_grid={'n_estimators':[100,250,500],
             'learning_rate':[0.01,0.5,0.1],
             'max_depth':[1,2,3,4,5],
             'subsample':[.5,.75],
             'random_state':[42]}

search=GridSearchCV(estimator=GBR, param_grid=search_grid, scoring='neg_mean_squared_error',cv=5)
search.fit(X_train,y_train)
search.best_params_

Best Parameters : {'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 500,
 'random_state': 42,
 'subsample': 0.75}
<br>
<br>
Using these 'Best parameters' model is overfitting too much

In [65]:
model = GradientBoostingRegressor(learning_rate=0.1, max_depth=2, n_estimators=1000, random_state=42, subsample=0.75)
model.fit(X_train, y_train)

print('Test Set --------------')
y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

print('Root Mean Square Error : ', rmse )
print('R2 Score : ', score )
print('\n')

print('Train Set --------------')
y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmse = np.sqrt(mse)
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
print('Root Mean Square Error : ', rmse ) 
print('R2 Score : ', score )
print("-------------------------------------------------------------------------------")

In [37]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05, 0.1],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

xgbr = xgb.XGBRegressor(seed =1)
GS_CV = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=3,cv=5)
GS_CV.fit(X_train, y_train)
print("Best parameters:", GS_CV.best_params_)
print("Lowest RMSE: ", (-GS_CV.best_score_)**(1/2.0))

Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 1000}
Lowest RMSE:  212.38552452115346
<br>
<br>
Using these 'Best parameters' model is overfitting too much

#### Tweaking max_depth and n_estimator manually to prevent overfitting

In [61]:
model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.1,max_depth=3,n_estimators=500)# XGBRegressor(learning_rate=0.1, max_depth=10, colsample_bytree=0.7)
model.fit(X_train, y_train)

print('Test Set --------------')
y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

print('Root Mean Square Error : ', rmse )
print('R2 Score : ', score )
print('\n')

print('Train Set --------------')
y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmse = np.sqrt(mse)
score = model.score(X_train,y_train)
R2 = r2_score( y_train, y_predictions )
print( " R2 Score : ", R2 )
RMSE_train.append(rmse)
print('Root Mean Square Error : ', rmse ) 
print("-------------------------------------------------------------------------------")

In [39]:
X_train

In [40]:
df_Test

# Submission
***

In [75]:
df_Test.drop(columns='title',inplace=True)

In [76]:
# Final model
final_model = RandomForestRegressor(bootstrap=True, max_depth=6, min_samples_leaf=4, min_samples_split=2, n_estimators=100)
final_model.fit(X_train,y_train)
final_predictions = final_model.predict(df_Test)

In [83]:
final_predictions_df = pd.DataFrame()
final_predictions_df['price1'] = final_predictions

In [85]:
# Checking if any SalePrice predicted is negative
final_predictions_df[final_predictions_df['price1']<=0]

In [84]:
Submission_Data = pd.concat([df_Test_id,final_predictions_df], axis='columns')
Submission_Data.head()

In [86]:
Submission_Data.to_csv('Summer_Analytics_2022_Hackathon_1_Shirsh_Submission_File.csv', encoding = 'utf-8-sig', index=False) 